# Algorithme génétique

## Les algorithmes évolutifs

### Algorithmes génétiques

Le Machine Learning peut apparaître complexe de prime abord, et pour cause certains des algorithmes qui constituent cet ensemble sont très loin d’être accessibles.

Ceci étant, il existe un type d’algorithme évolutif plus simple à saisir parmi ceux-ci: les algorithmes génétiques (ou AG).

### Principes

Comme leur nom l'indique, les AGs ont certains points communs avec la génétique en biologie.

Un individu est un ensemble de données concaténées, chacune de ses données représente un gène.

L'ensemble des individus représente la population, tandis que chacune des populations (une par étape de l'algorithme est une génération.

D'une génération à l'autre, l'individu le plus apte à répondre à la problématique est conservé tandis que les autres sont obtenus via le croisement des individus de la génération précédente. Au terme de ces croisements, des modifications peuvent avoir lieu: les mutations.

## Comment faire?

Au final, les AGs ne sont pas particulièrement compliqués à implémenter, et ce TD vous permettra de créer un exemple simple.

**Attention: Le contenu que vous produirez ici sera réutilisé lors du TD Examen AG, qui sera noté. Veillez à la modularité de votre code.**

**Egalement, pensez à commenter votre code et à le rendre lisible (voir la norme PEP8)**

### Objectif

A la fin de ce TD, votre code sera capable de trouver, de lui-même, une chaîne de caractères définie. Bien sûr, l'utilité est faible, mais il s'agit de pouvoir traiter ensuite toute sorte de données, comme par exemple, utiliser l'AG pour estimer une courbe à partir d'un ensemble de points fournis.

Pour ce TD, le code devra retrouver la chaîne de caractères "Hello World".

### Description

Pour fonctionner, l'AG a besoin:
- d'une fonction d'évaluation permettant d'indiquer la proximité ou l'erreur entre l'individu et la solution optimale. Dans notre cas, la distance entre chaque caractère sera utilisée, la fonction d'évaluation pour une distance optimale vaudra donc 0.
- d'une fonction de croisement qui permet d'obtenir une nouvelle génération à partir d'une génération pré-existante, prenant une partie de deux individus pour en former un nouveau, et ce pour l'ensemble de la population.
- de paramètres de mutation, indiquant les chances pour un nouvel individu d'être altéré, et la force de cette altération.

### Avant le code

In [1]:
# tous les imports de ce TD devront être placés ici
import random
import time
# le mot à trouver
test_length = 10
target = "Hello World Hello Wo"

### Création des méthodes

#### Calcul de distance et localisation du meilleur individu

Avant toute chose, il faut réfléchir à la manière de représenter nos données.

Nous traiterons des chaînes de caractères. En Python, une chaîne de caractères fonctionne (presque) exactement comme une liste de caractères, et ces caractères peuvent être remplacés par leur code ASCII, qui est numérique.

Comme dit précédemment, nous allons définir notre individu cible comme la liste des codes ASCII de "H", "e", "l", "l", ... "d", qui aura comme distance 0.

Puisque nous avons les codes ASCII des caractères à disposition, il est simple de trouver, pour chaque indice de la liste, la distance entre le caractère cible et le caractère courant, qui n'est rien de plus qu'une simple différence. De même, la distance globale de la chaîne est une somme des distances.

Une fois les distances d'un ensemble de chaînes obtenues, celles-ci peuvent être ordonnées.

In [2]:
# la fonction de conversion d'une chaîne de caractères en liste de valeurs ASCII vous est founie
def string_to_int_list(string):
    return [ord(character) for character in list(string)]

##### Exo 1:
- **Créez la méthode permettant de calculer la distance entre un mot et un autre**
- **Vérifiez que la distance avec "Hello World" est correcte pour les mots suivants: "COjsy OfUkp" (105) et "Hemlo Wohld" (11)**

In [9]:
# la fonction qui calcul la distance entre deux mots
def get_distance(mot, target):
    string_ascii = string_to_int_list(mot) #conversion du mot en ascii
    target_ascii = string_to_int_list(target)
    dist = 0 #distance entre les deux motes
    for i in range (0, len(mot)): #boucle pour calculer la distance
        dist += abs(string_ascii[i] - target_ascii[i])
    return dist #retourne de la distance
    
    
print(get_distance("Genetiquj", 'Genetique'))


5


Maintenant qu'il est possible d'attribuer une valeur de distance entre deux mots, nous pouvons ordonner des mots grâce à cette valeur.

##### Exo 2:
- **Créez la méthode permettant de retrouver le mot le plus proche d'une cible et la valeur de cette distance**
- **Testez avec une liste de mots définie par vos soins, et la cible "Hello World"**

In [19]:
# la fonction qui determine le mot le plus proche d'un autre dans une liste    
def getBest(liste, target):
    dist = 0
    i = 0
    for item in liste:
        if i == 0:
            i += 1
            dist = get_distance(item, target)
            mot = item
        if get_distance(item, target) < dist:
            dist = get_distance(item, target)
            mot = item
            i += 1
    return mot

liste = ["Helsa Worrd", "AZERT QWERT", "hello world", "World Hello", "Hella World"]

getBest(liste, target)

'hello world'

#### Création d'une première génération

Bien que nous connaissions le mot cible, nous partirons d'une population constituée d'un ensemble de mots générés aléatoirement.

Gardez à l'esprit que la taille de votre population définira la vitesse d'exécution de votre code. Ainsi, bien qu'un ensemble d'individus important  aura la diversité pour atteindre rapidement (en termes d'itérations) la solution, un ensemble plus restreint passera chacune des itérations rapidement.

##### Exo 3:
- **Créez la méthode d'initialisation d'une liste de mots aléatoires**
- **Utilisez cette méthode pour générer aléatoirement un ensemble de mots, et comparez-les à la cible.**

In [20]:
def word_list_init():
    listSize = 400 #random.randint(50,500)
    listeMots = [""] * listSize
    for i in range(0, listSize):
        for j in range (test_length):
            listeMots[i] = listeMots[i] + chr(random.randint(0, 255))
    return listeMots

liste = word_list_init()
#for mot in liste:
    #print(mot, "-> distance ->", get_distance(mot, target))


#### Passage d'une génération à une autre

Les opérations de base sont maintenant établies, il est temps de rendre notre système évolutif.

D'une génération à l'autre, les individus doivent évoluer. Mais, il n'est pas certain que cette évolution se fasse dans la direction espérée.

La théorie de l'évolution dans le domaine biologique indique que l'espèce la plus adaptée à son environnement survivra plus probablement que d'autres. Dans notre cas, nous pouvons discerner l'individu le plus proche de la solution et le conserver (il "survit"). Ainsi, le meilleur individu d'une génération ne peut pas être moins adapté que celui d'une génération passée (principe de non-régression).

Le reste de la population de la nouvelle génération est produite comme dans le cas de la biologie. Deux parties complémentaires (chromosomes) de deux individus (parents) seront combinées pour obtenir un nouvel individu (enfant).

##### Exo 4:
- **Créez la méthode de transition entre deux générations. Notez que:**
 - **Le meilleur individu (ou les x meilleurs individus) devrait être conservé.**
 - **Les individus restants devraient être un croisement d'individus de la liste des mots précédente (meilleur mot précédent compris).**
 - **La liste de mots résultante devrait être de même longueur que la précédente.**
- **Appliquez la méthode de manière itérative, en indiquant à chaque fois le meilleur élément de la génération et la distance avec la cible.**
- **Au bout d'un certain nombre d'itérations, que se passe-t-il?**

In [23]:
def crossover(mot1, mot2):
    mot3 = ""
    for i in range(0, len(mot1)): #Je parcours le mot (Enfant)
        unDeux = random.randint(1, 2) #De quel parent j'herite de la char
        if unDeux == 1:
            mot3 = mot3 + mot1[i] #Parent 1
        else:
            mot3 = mot3 + mot2[i] #Parent 2
    return mot3

def new_generation(liste):
    old_generation = liste
    old_generation.sort(key=lambda x: get_distance(x, target))
    newGen = [""] * len(liste) #Je crée la liste de la nouvelle generation
    percent = round(len(old_generation)*0.1)
    for i in range (0,percent):
        newGen[i] = old_generation[i] #Je prend la meilleur mot de la generation precedente  (survivant)
    for i in range(percent, len(old_generation)): #Je parcours la nouvelle generation
        newGen[i] = crossover(old_generation[random.randint(0, len(liste) - 1)], old_generation[random.randint(0, len(old_generation) - 1)]) #Crossover
    return mutation(newGen)

def evolution(nbGen):
    liste = word_list_init()
    print("Gen 0")
    print("Meilleur ->", getBest(liste, target), "-> Distance -> ", get_distance(getBest(liste, target), target))
    gen = 0
    for i in range(1, nbGen):
        liste = new_generation(liste)
        gen = i
    print("Gen", gen)
    print("Meilleur ->", getBest(liste, target), "-> Distance -> ", get_distance(getBest(liste, target), target))
    
evolution(100)

Gen 0
Meilleur -> £Â»\lÊûú´] ] -> Distance ->  1092
Gen 99
Meilleur -> ÕPÊwH3Ðå gÎE<XÈ(& -> Distance ->  64


#### Créer la diversité génétique

Le principe de mutation permet d'ajouter de la diversité lors de l'évolution de l'AG. En effet, avec une faible population, il est presque certain que le croisement normal d'éléments finisse dans une impasse, on parle de minimum local.

Dans notre cas, la notion de minimum local n'a pas véritable lieu d'être. Il nous suffit d'ajouter de nouveaux éléments innovants pour permettre à l'AG de reprendre son évolution.

Cette innovation prend la forme d'une altération d'un (ou plusieurs) caractère(s) d'un individu lors de sa création pour une nouvelle génération. Pour s'assurer que l'on ne dégrade pas le meilleur individu, il est conseillé de lui éviter cette étape.

La mutation est définie suivant deux paramètres principaux. Tout d'abord, sa probabilité, entre 0 et 1, définit sa fréquence. Aussi, sa force définit l'effet de la mutation, et peut prendre n'importe quelle forme, de +1/-1 à une réaffectation aléatoire. Cette mutation peut s'appliquer lettre par lettre, sur le mot entier, ou sur un ensemble de lettres aléatoires.

##### Exo final:
- **Créez une méthode permettant de définir le procédé de mutation**
- **Utilisez cette méthode pour obtenir la terminaison de votre AG (réduire la distance du meilleur mot à 0)**
- **Votre code est complet, faites varier certains de vos paramètres afin de le rendre plus efficace, si vous le souhaitez**
- **Rendez votre code adaptatif, capable d'acception diverses longueurs de chaînes de caractères.**

In [24]:
def mutation(generation):  
    for i in range (0,round(len(generation)*0.3)):
        rand_word = random.randint(round(len(generation)*0.1),len(generation)-1)
        rand_letter = random.randint(0,len(generation[rand_word])-1)
        motList = list(generation[rand_word])
        motList[rand_letter] = chr(random.randint(0, 255))
        mot = ""
        for lettre in motList:
            mot = mot + lettre
        generation[rand_word] = mot
    return generation

#Optimiser le code en variant la force en fonction de la distance
#print(mutation("Hella World Hello Wo"))



Le bloc suivant vous permet de tester l'exécution de votre code dans les conditions de test finales. Les paramètres des méthodes "word_list_init", "get_distance" et "new_generation" sont à compléter.

La valeur de la variable "test_length" sera modifiée lors de l'évaluation du code.

In [25]:
# espace de test et d'exécution final
test_length = 20
target = "".join([chr(random.randint(0, 255)) for _ in range(test_length)])
start = time.time()
ended_amount = 0

while time.time() - start < 180:
    population = word_list_init()
    distance = get_distance(getBest(population, target), target)

    while distance > 0:
        population = new_generation(population)
        distance = get_distance(getBest(population, target), target)
        #print(distance)
        if time.time() - start > 180:
            break
    
    ended_amount += 1
    print("Finished ", ended_amount, "times.") 

print("After 15 minutes, finished", ended_amount, "times and stopped with a distance of", distance)

Finished  1 times.
Finished  2 times.
Finished  3 times.
Finished  4 times.
Finished  5 times.
Finished  6 times.
Finished  7 times.
Finished  8 times.
Finished  9 times.
Finished  10 times.
Finished  11 times.
Finished  12 times.
Finished  13 times.
Finished  14 times.
Finished  15 times.
Finished  16 times.
Finished  17 times.
Finished  18 times.
After 15 minutes, finished 18 times and stopped with a distance of 9


IMPORTANT:

Ce TD est noté (note bonus).
L'évaluation sera effectuée par le redémarrage du noyau et l'exécution complète du code. Vérifiez la validité de votre travail avec "Noyau" -> "Redémarrer & tout exécuter". Tout code ne fonctionnant pas en suivant cette procédure vaudra 0.

Le barème est le suivant:
- **L'exécution complète attribuera au plus 12 points.** 2 points sont attribués pour chaque méthode correctement implémentée.
- **Les codes terminant seront mis en compétition** 0 à 6 points seront attribués en fonction du classement.
- **La propreté** (respect du PEP8) **vaudra 2 points.** Un code non propre peut faire perdre jusqu'à 3 points.
- Le code doit respecter le côté aléatoire du sujet. Cela inclut la génération initiale, le croisement et la mutation. Toute méthode parmi les trois indiquées ne respectant pas ce point vaudra 0.

Attention, tout jour de retard pour le rendu de ce travail entraînera une pénalité de 5 points.

Aussi ce code devant être utilisé pour le TD Examen AG, il est conseillé d'y mettre du soin.

Le rendu prend la forme de ce notebook, à envoyer par mail.